In [155]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [156]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [157]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [158]:
%%capture pwd
!pwd

In [159]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('is-the-uk-on-track-to-meet-its-climate-commitments', 300, 500)

# Fig 1

In [160]:
df = pd.read_excel("raw/EO_Article_-_UK_targets_Fig_1.xlsx", sheet_name="Graph")
df = df.set_index("Unnamed: 0").T.drop("2030 Target", axis=1).reset_index()
df.columns = ["year", "historic", "projected"]

In [161]:
f = "fig1_projected-emissions"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,historic,projected
0,1990,793.78739,NaN
1,1991,802.64404,NaN
2,1992,781.03397,NaN
3,1993,761.13853,NaN
4,1994,751.96395,NaN


In [162]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="⬅ historical   projection ➡",
            titleY=-15,
            titleX=302,
            titleFontWeight="normal",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            titleFontSize=10,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-dark-blue"])
    .encode(
        y=alt.Y(
            "historic:Q",
            axis=alt.Axis(
                grid=True,
                gridOpacity=0.1,
                gridColor=colors["eco-gray"],
                title="GtCO₂e per year",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=25,
                titleY=28,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
        )
    )
    .transform_filter("datum.year<=2020")
)
area = (
    base.mark_line(color=colors["eco-turquiose"])
    .encode(y=alt.Y("projected:Q"))
    .transform_filter("datum.year>=2020")
)
ruler = (
    base.mark_line(color=colors["eco-gray"])
    .encode(y="y:Q")
    .transform_calculate(y="254")
)
label = (
    alt.Chart(pd.DataFrame([{"x": 2000, "y": 280, "t": "2030 target"}]))
    .mark_text(color=colors["eco-gray"], fontSize=10)
    .encode(text="t:N", x="x:Q", y="y:Q")
)
r2020 = (
    base.mark_rule(
        color=scale_lightness(colors["eco-gray"], 1.5),
        strokeDash=[5, 5],
        strokeWidth=0.5,
    )
    .encode(x="x:Q")
    .transform_calculate(x="2020")
)
title = alt.TitleParams(
    "UK emissions up to date and projected reduction to 2030",
    subtitle=["Source: BEIS (2020)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((ruler + r2020 + area + line + label).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

In [163]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="⬅ historical   projection ➡",
            titleY=-15,
            titleX=302,
            titleFontSize=12,
            labelFontSize=12,
            titleFontWeight="normal",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
line = (
    base.mark_line(color=colors["eco-mid-blue"])
    .encode(
        y=alt.Y(
            "historic:Q",
            axis=alt.Axis(
                gridOpacity=0.1,
                gridColor=colors["eco-gray"],
                title="GtCO₂e per year",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=31,
                titleY=30.5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
        )
    )
    .transform_filter("datum.year<=2020")
)
area = (
    base.mark_line(color=colors["eco-turquiose"])
    .encode(y=alt.Y("projected:Q"))
    .transform_filter("datum.year>=2020")
)
ruler = (
    base.mark_line(color=colors["eco-gray"])
    .encode(y="y:Q")
    .transform_calculate(y="254")
)
label = (
    alt.Chart(pd.DataFrame([{"x": 2000, "y": 290, "t": "2030 target"}]))
    .mark_text(color=colors["eco-gray"], fontSize=12)
    .encode(text="t:N", x="x:Q", y="y:Q")
)
r2020 = (
    base.mark_rule(
        color=scale_lightness(colors["eco-gray"], 0.7),
        strokeDash=[5, 5],
        strokeWidth=0.5,
    )
    .encode(x="x:Q")
    .transform_calculate(x="2020")
)
title = alt.TitleParams(
    "UK emissions up to date and projected reduction to 2030",
    subtitle=["Source: BEIS (2020)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    ((ruler + r2020 + area + line + label).properties(height=300, width=400))
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png", scale_factor=2.0)
layer1.save("visualisation/" + f + "_dark.svg")
readme = re.sub(f, f + "_dark", readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [164]:
configSource = "raw/chartENV11.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"]).drop("Detail", axis=1)

In [165]:
f = "fig2-patents"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = (
    "### "
    + f
    + " --- [json](visualisation/"
    + f
    + '.json "'
    + f
    + '") [svg](visualisation/'
    + f
    + '.svg "'
    + f
    + '") [png](visualisation/'
    + f
    + '.png "'
    + f
    + '")\n[!["'
    + f
    + '"](visualisation/'
    + f
    + '.svg "'
    + f
    + '")](visualisation/'
    + f
    + '.svg "'
    + f
    + '")\n\n'
)
df.head()

,Time,Country,Value
0,2000,Canada,7.82
1,2001,Canada,7.68
2,2002,Canada,8.48
3,2003,Canada,9.51
4,2004,Canada,9.38


In [166]:
config = json.loads(open(configSource, "r").read())
if not LOCAL:
    config["data"]["url"] = f2
config["encoding"]["x"]["axis"]["grid"] = False
config["encoding"]["x"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["x"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["grid"] = True
config["encoding"]["y"]["axis"]["gridColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["gridOpacity"] = 0.1
config["encoding"]["y"]["axis"]["title"] = ""
config["encoding"]["y"]["axis"]["format"] = ".0%"
config["encoding"]["y"]["axis"]["titleAnchor"] = "start"
config["encoding"]["y"]["axis"]["labelColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["tickColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["domainColor"] = colors["eco-gray"]
config["encoding"]["y"]["axis"]["titleFontSize"] = 10
config["encoding"]["y"]["axis"]["titleFontWeight"] = "normal"
config["encoding"]["y"]["axis"]["ticks"] = False
config["encoding"]["y"]["axis"]["labelAlign"] = "left"
config["encoding"]["y"]["axis"]["labelBaseline"] = "middle"
config["encoding"]["y"]["axis"]["labelPadding"] = -5
config["encoding"]["y"]["axis"]["labelOffset"] = -10
config["encoding"]["y"]["axis"]["titleBaseline"] = "bottom"
config["encoding"]["y"]["axis"]["titleAngle"] = 0
config["encoding"]["y"]["axis"]["titleAlign"] = "left"
config["encoding"]["y"]["axis"]["tickCount"] = 7
config["encoding"]["y"]["axis"]["titleX"] = 45
config["encoding"]["y"]["axis"]["titleY"] = -5
config["title"]["dy"] = -5
config.pop("$schema")
config["encoding"]["color"]["legend"] = None
config["encoding"]["y"]["scale"]["domain"] = [
    config["encoding"]["y"]["scale"]["domain"][0] / 100,
    config["encoding"]["y"]["scale"]["domain"][1] / 100,
]
config["encoding"]["y"]["field"] = "m"
config["view"] = {"stroke": None}
config["width"] = 300
config["height"] = 220
base = alt.Chart.from_dict(config).transform_calculate(m="datum.Value/100")
text = (
    base.mark_text(dx=5, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Time)==2018")
    .transform_filter("datum.Country!='Italy'")
    .transform_filter("datum.Country!='Canada'")
    .transform_filter("datum.Country!='Great Britain'")
    .transform_filter("datum.Country!='United States'")
)
text2 = (
    base.mark_text(dx=5, dy=13, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Time)==2018")
    .transform_filter("datum.Country=='Italy'")
)
text3 = (
    base.mark_text(dx=40, dy=0, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Time)==2018")
    .transform_filter("datum.Country=='Canada'")
)
text4 = (
    base.mark_text(dx=5, dy=-3, align="left", text="United Kingdom")
    .transform_filter("year(datum.Time)==2018")
    .transform_filter("datum.Country=='Great Britain'")
)
text5 = (
    base.mark_text(dx=5, dy=2, align="left")
    .encode(text="Country:N")
    .transform_filter("year(datum.Time)==2018")
    .transform_filter("datum.Country=='United States'")
)
layer1 = (base + text + text2 + text3 + text4 + text5).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1
# altair_viewer.display(config, inline=True)

alt.LayerChart(...)

In [167]:
theme = "_dark"
base.title.fontSize = 14
base.title.subtitleFontSize = 12
base.title.dy -= 2
base.encoding.y.axis.titleY += 2
base.encoding.y.axis.titleX += 8
base.title.color = colors["eco-dot"]
base.title.subtitleColor = colors["eco-dot"]
layer1 = (
    (base + text + text2 + text3 + text4 + text5)
    .transform_filter("datum.Country!='World'")
    .configure_view(stroke=None)
)
layer1 = layer1.configure_axisY(labelFontSize=12)
layer1 = layer1.configure_axisX(labelFontSize=12)
layer1.config.font = "Georgia"
layer1.config.background = colors["eco-background"]
layer1.config.view.stroke = None
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)